In [52]:
# Importing libraries
import pandas as pd

In [53]:
filenames=["6_3","06_9","06_16 ","06_16"]
raw_df=pd.DataFrame()
for file in filenames:
    a=pd.read_json("datasets/{}.json".format(file),lines=True)
    print(a.shape)
    raw_df = pd.concat([raw_df,pd.read_json("datasets/{}.json".format(file),lines=True)])

(164, 4)
(54, 4)
(44, 4)
(24, 4)


In [54]:
raw_df.reset_index(inplace=True)
raw_df.drop(columns=["index"],inplace=True)

We clearly see that the username in each case is always "admin" for all the rows

In [55]:
raw_df

,doc_id,text,entities,username
0,1835,1 The applicants Sharman Networks Ltd ('Sharma...,"[[17, 37, Organization], [38, 58, Organization...",admin
1,1836,"When referring to the applicants generally, I ...","[[60, 84, person]]",admin
2,1837,Each of the Sharman applicants was one of ten ...,"[[12, 30, person], [134, 157, Organization], [...",admin
3,1838,Wilcox J made orders ancillary to the Mareva o...,"[[55, 68, Date], [91, 109, person], [0, 8, Jud...",admin
4,1839,2 Wilcox J delivered judgment on the complex i...,"[[103, 119, Date], [122, 186, Versus], [2, 10,...",admin
...,...,...,...,...
281,3160,I think that is sensible given that it is self...,"[[77, 86, Location], [123, 137, person]]",admin
282,3161,"Moreover, once removed from Australia, such en...","[[28, 38, Location], [83, 93, Location]]",admin
283,3162,12 I therefore propose to make an order restra...,"[[107, 116, Location]]",admin
284,3165,If he is to be removed from Australia notwiths...,"[[28, 37, Location]]",admin


In [56]:
raw_df["username"].unique()

array(['admin'], dtype=object)

In [57]:
raw_df.drop(columns=["username"],inplace=True)

In [58]:
raw_df["doc_id"].nunique()

262

In [59]:
raw_df["doc_id"].value_counts()

3136    2
3165    2
3161    2
3155    2
3153    2
       ..
2000    1
2001    1
2002    1
2003    1
3214    1
Name: doc_id, Length: 262, dtype: int64

In [60]:
raw_df["text"].nunique()

262

In [61]:
raw_df["text"].value_counts()

Both agree that the removal of the applicant from Australia may cause his mental condition to deteriorate.                                                                                                                                                                                                                                                                                                                                                                                                                                2
If he is to be removed from Australia notwithstanding his mental condition, that removal should take place promptly.                                                                                                                                                                                                                                                                                                                                                                        

In [62]:
raw_df.drop_duplicates(subset=["doc_id"],inplace=True)

In [63]:
raw_df

,doc_id,text,entities
0,1835,1 The applicants Sharman Networks Ltd ('Sharma...,"[[17, 37, Organization], [38, 58, Organization..."
1,1836,"When referring to the applicants generally, I ...","[[60, 84, person]]"
2,1837,Each of the Sharman applicants was one of ten ...,"[[12, 30, person], [134, 157, Organization], [..."
3,1838,Wilcox J made orders ancillary to the Mareva o...,"[[55, 68, Date], [91, 109, person], [0, 8, Jud..."
4,1839,2 Wilcox J delivered judgment on the complex i...,"[[103, 119, Date], [122, 186, Versus], [2, 10,..."
...,...,...,...
257,3208,The short answer to this argument is that it w...,"[[120, 126, Location]]"
258,3209,"To the contrary, he indicated in his applicati...","[[104, 110, Location]]"
259,3211,While it appears his position on this issue wa...,"[[175, 181, Location]]"
260,3212,Even if the Tribunal erred in construing the t...,"[[194, 200, Location]]"


In [64]:
raw_df["text"]=raw_df["text"].astype("string")

In [65]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262 entries, 0 to 261
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   doc_id    262 non-null    int64 
 1   text      262 non-null    string
 2   entities  262 non-null    object
dtypes: int64(1), object(1), string(1)
memory usage: 8.2+ KB


In [66]:
raw_df

,doc_id,text,entities
0,1835,1 The applicants Sharman Networks Ltd ('Sharma...,"[[17, 37, Organization], [38, 58, Organization..."
1,1836,"When referring to the applicants generally, I ...","[[60, 84, person]]"
2,1837,Each of the Sharman applicants was one of ten ...,"[[12, 30, person], [134, 157, Organization], [..."
3,1838,Wilcox J made orders ancillary to the Mareva o...,"[[55, 68, Date], [91, 109, person], [0, 8, Jud..."
4,1839,2 Wilcox J delivered judgment on the complex i...,"[[103, 119, Date], [122, 186, Versus], [2, 10,..."
...,...,...,...
257,3208,The short answer to this argument is that it w...,"[[120, 126, Location]]"
258,3209,"To the contrary, he indicated in his applicati...","[[104, 110, Location]]"
259,3211,While it appears his position on this issue wa...,"[[175, 181, Location]]"
260,3212,Even if the Tribunal erred in construing the t...,"[[194, 200, Location]]"


In [67]:
# Converting into BIO format
def convert_to_bio(text, entities):
    words = text.split()
    bio_tags = ["O"] * len(words)
    for start, end, entity_type in entities:
        s = text[start:end]
        for i,word in enumerate(words):
            if word is s[0]:
                bio_tags[i] = entity_type
            elif word in s:
                bio_tags[i] = entity_type
    return words, bio_tags

# Create empty lists for tokens and tags
tokens_list = []
tags_list = []

# Iterate through the rows and convert to BIO format
for index, row in raw_df.iterrows():
    tokens, tags = convert_to_bio(row["text"], row["entities"])
    tokens_list.append(tokens)
    tags_list.append(tags)

# Add the lists to the DataFrame
raw_df["tokens"] = tokens_list
raw_df["tags"] = tags_list

In [76]:
try_df = raw_df.drop(columns=["doc_id","tokens","entities"])

In [94]:
for i,row in raw_df.iterrows():
    print(len(raw_df["tokens"][i]),len(raw_df["tags"][i]))

40 40
14 14
42 42
56 56
35 35
105 105
36 36
53 53
63 63
20 20
37 37
33 33
15 15
42 42
80 80
67 67
26 26
22 22
42 42
52 52
45 45
78 78
44 44
47 47
13 13
49 49
64 64
68 68
48 48
67 67
29 29
69 69
80 80
47 47
53 53
36 36
22 22
42 42
57 57
45 45
65 65
71 71
53 53
20 20
47 47
48 48
42 42
36 36
44 44
33 33
34 34
33 33
75 75
67 67
64 64
51 51
26 26
26 26
50 50
41 41
38 38
14 14
26 26
59 59
36 36
42 42
34 34
43 43
35 35
24 24
101 101
37 37
38 38
14 14
22 22
52 52
64 64
57 57
55 55
39 39
34 34
28 28
57 57
37 37
79 79
45 45
15 15
59 59
41 41
57 57
110 110
91 91
82 82
21 21
30 30
38 38
30 30
24 24
92 92
20 20
40 40
30 30
38 38
57 57
99 99
63 63
38 38
67 67
53 53
43 43
38 38
41 41
65 65
56 56
12 12
58 58
40 40
26 26
7 7
10 10
15 15
21 21
11 11
7 7
10 10
8 8
110 110
22 22
25 25
29 29
65 65
44 44
5 5
10 10
10 10
12 12
13 13
30 30
47 47
14 14
39 39
58 58
19 19
92 92
36 36
30 30
14 14
19 19
52 52
21 21
21 21
26 26
33 33
24 24
18 18
19 19
57 57
21 21
17 17
24 24
43 43
44 44
29 29
24 24
43 43
10 10
58 5

In [97]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class NERDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        labels = self.data.iloc[idx]['tags']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=False,
            return_attention_mask=True,
            pad_to_max_length=True,
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        labels = [labels_map[tag] for tag in labels]

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Map labels to numerical IDs
labels_map = {'O': 0, 'Organization': 1, 'person': 2, 'Date': 3, 'Judges': 4, 'Versus': 5, 'Location': 6, 'Act&Regulation':7,'Acts&Regulation':8, 'Person':9}

# Create the NERDataset
dataset = NERDataset(try_df, tokenizer)


In [100]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments

# Load the pre-trained BERT model
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_map))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./ner_model',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    evaluation_strategy='epoch',
    save_total_limit=1,
)


# Create a Trainer instance for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # You can specify a data collator if needed
    train_dataset=dataset
)

# Fine-tune the model
trainer.train()
trainer.save_model()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/51 [00:00<?, ?it/s]

ValueError: expected sequence of length 14 at dim 1 (got 63)

In [99]:
def readfile(filename):
    f = open(filename)
    data = []
    sentence = []
    label= []
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                data.append((sentence,label))
                sentence = []
                label = []
            continue
        splits = line.split(' ')
        sentence.append(splits[0])
        label.append(splits[-1][:-1])

    if len(sentence) >0:
        data.append((sentence,label))
        sentence = []
        label = []
    return data

In [125]:
train_data = readfile("datasets/legal_train.txt")

In [101]:
test_data = readfile("datasets/legal_test.txt")

In [103]:
valid_data = readfile("datasets/legal_valid.txt")